### Calculate average similarity score of grund truth

In [1]:
import pandas as pd
df = pd.read_csv('./data/ground_truth_test.csv')
print(df.columns)

Index(['topic', 'argument', 'key_point', 'stance', 'label',
       'cos_similarity_bert_embedding(gpt4)', 'gpt4', 'llama2', 'cos(llama2)'],
      dtype='object')


In [ ]:
df_list = df.values.tolist()
print(df_list[0])
print(len(df_list))

In [ ]:
new_header = ['cos_similarity_bert_embedding']
if 'cos_similarity_bert_embedding' not in df.columns:
    df = pd.concat([df, pd.DataFrame(columns=new_header)], axis=1)
    print(df.columns)
    df.to_csv('./data/ground_truth_test.csv', index=False)
    print("New header added successfully.")

In [11]:
from transformers import BertTokenizer, BertModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def calculate_bert_similarity(sentence1, sentence2):
    # Load pre-trained BERT model and tokenizer
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    model = BertModel.from_pretrained('bert-base-uncased')

    # Tokenize and obtain embeddings for each sentence
    tokens1 = tokenizer(sentence1, return_tensors='pt')
    tokens2 = tokenizer(sentence2, return_tensors='pt')

    with torch.no_grad():
        outputs1 = model(**tokens1)
        outputs2 = model(**tokens2)

    # Extract the embeddings from the model outputs
    embedding1 = outputs1.last_hidden_state.mean(dim=1).numpy()
    embedding2 = outputs2.last_hidden_state.mean(dim=1).numpy()

    # Reshape embeddings for cosine similarity calculation
    embedding1 = np.squeeze(embedding1)
    embedding2 = np.squeeze(embedding2)

    # Calculate cosine similarity
    similarity_score = cosine_similarity([embedding1], [embedding2])[0][0]

    return similarity_score

In [ ]:
import os

# Set OMP_NUM_THREADS to 1
os.environ["OMP_NUM_THREADS"] = "1"

# Verify the setting
print("OMP_NUM_THREADS:", os.environ.get("OMP_NUM_THREADS"))

In [13]:
df_list = df_list[:500] # [0:1000) = [0, 999] = length:1000
for id, item in enumerate(df_list):
    score = calculate_bert_similarity(item[1],item[2])
    df_id = id+0 # the added num is the left boundary of df_list
    print(str(df_id) + ": " + str(score))
    df.at[df_id,'cos_similarity_bert_embedding'] = score

df.to_csv('./data/ground_truth_test.csv', index=False)
print("Add new data successfully!!!!!")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


7: 0.8382052


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


8: 0.8183581
Add new data successfully!!!!!


In [2]:
import pandas as pd
gt_df = pd.read_csv('./data/ground_truth_test.csv')
print(gt_df.columns)

Index(['topic', 'argument', 'key_point', 'stance', 'label',
       'cos_similarity_bert_embedding(gpt4)', 'gpt4', 'llama2', 'qwen'],
      dtype='object')


In [4]:
cos = gt_df['llama2'].values.tolist()
average_score = sum(cos)/len(cos)
print(average_score)

0.7213153153153465
